드라이브에서 받으신 data.zip 안의 내용물을 week4/data안에 위치시켜 주세요

In [2]:
# import autoreload
# %load_ext autoreload
# %autoreload 2
import sys
import pandas as pd
sys.path.append('.')
from data_utils import refine_and_save_events

refine_and_save_events(dataset_name='England')

England                : 100%|██████████| 380/380 [00:31<00:00, 12.06it/s]


In [6]:
match_events = pd.read_pickle('data/refined_events/England/2499719.pkl')
match_events

,match_id,event_id,period,time,team_id,team_name,player_id,player_name,event_type,sub_event_type,tags,start_x,start_y,end_x,end_y
0,2499719,177959171,1H,2.759,1609,Arsenal,25413,A. Lacazette,Pass,Simple pass,[Accurate],50.96,34.68,32.24,14.96
1,2499719,177959172,1H,4.947,1609,Arsenal,370224,R. Holding,Pass,High pass,[Accurate],32.24,14.96,53.04,17.00
2,2499719,177959173,1H,6.542,1609,Arsenal,3319,M. Özil,Pass,Head pass,[Accurate],53.04,17.00,36.40,19.72
3,2499719,177959174,1H,8.143,1609,Arsenal,120339,Mohamed Elneny,Pass,Head pass,[Accurate],36.40,19.72,42.64,3.40
4,2499719,177959175,1H,10.302,1609,Arsenal,167145,Bellerín,Pass,Simple pass,[Accurate],42.64,3.40,74.88,8.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1775,2499719,177961040,2H,2992.492,1631,Leicester City,8480,K. Schmeichel,Pass,Launch,[Not accurate],7.28,31.96,47.84,34.68
1776,2499719,177961035,2H,2994.901,1609,Arsenal,49876,G. Xhaka,Pass,Head pass,[Accurate],56.16,33.32,75.92,28.56
1777,2499719,177961036,2H,2997.086,1609,Arsenal,7870,A. Ramsey,Shot,Shot,"[Right foot, Opportunity, Position: Out high r...",75.92,28.56,104.00,34.00
1778,2499719,177961041,2H,2998.964,1631,Leicester City,0,NaN,Interruption,Ball out of the field,[],3.12,35.36,NaN,NaN


In [ ]:
match_events.event_type.value_counts()

In [ ]:
match_events[match_events.event_type == "Save attempt"]["sub_event_type"].value_counts()